# Bulk processor

In [ ]:
from collections import namedtuple
from copy import deepcopy

from ssscoring import BREAKOFF_ALTITUDE
from ssscoring import PERFORMANCE_WINDOW_LENGTH
from ssscoring import aggregateResults
from ssscoring import convertFlySight2SSScoring
from ssscoring import dropNonSkydiveDataFrom
from ssscoring import getAllSpeedJumpFilesFrom
from ssscoring import getSpeedSkydiveFrom
from ssscoring import isValidJump
from ssscoring import jumpAnalysisTable
from ssscoring import processAllJumpFiles
from ssscoring import roundedAggregateResults
from ssscoring import totalResultsFrom
from ssscoring import validFlySightHeaderIn
from ssscoring.notebook import SPEED_COLORS
from ssscoring.notebook import graphAltitude
from ssscoring.notebook import graphAngle
from ssscoring.notebook import graphJumpResult
from ssscoring.notebook import initializeExtraYRanges
from ssscoring.notebook import initializePlot

import csv
import os
import os.path as path

import bokeh.plotting as bp
import bokeh.models as bm
import ipywidgets as widgets
import pandas as pd

In [ ]:
DATA_LAKE_ROOT = './data'

In [ ]:
jumpFiles = getAllSpeedJumpFilesFrom(DATA_LAKE_ROOT)

In [ ]:
jumpResults = processAllJumpFiles(jumpFiles)

---
## Results

In [ ]:
aggregate = aggregateResults(jumpResults)
sumResults = totalResultsFrom(aggregate)
display(aggregate)

### Rounded results for training log

In [ ]:
roundedResults = roundedAggregateResults(jumpResults)

### All jumps

In [ ]:
def dumpSimple(table: pd.DataFrame):
    fileName = os.path.join(DATA_LAKE_ROOT, resultRef+'.tsv')
    simpleTable = table.copy()
    simpleTable = simpleTable.drop('speedAngle', axis = 1)
    simpleTable.to_csv(fileName, sep = '\t', index = False)

In [ ]:
def displayJumpDataIn(resultsTable: pd.DataFrame):
    table = resultsTable.copy()
    # Experimental
    # For more information on the `interpolate` method and its options, see the [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.interpolate.html).
    # Additionally, you can also consider using other libraries like `scipy` which provides more advanced interpolation methods, such as `interp1d` or `griddata`. See the [scipy documentation](https://docs.scipy.org/doc/scipy/reference/interpolate.html) for more information.
    table.vKMh = table.vKMh.apply(round)
    table.hKMh = table.hKMh.apply(round)
    table['altitude (ft)'] = table['altitude (ft)'].apply(round)
    table.netVectorKMh = table.netVectorKMh.apply(round)
    table.index = ['']*len(table)
    display(table)
    # dumpSimple(table)

In [ ]:
allJumpsPlot = initializePlot('All jumps in set')
jumpNumber = 0
mixColor = 0
for resultRef in sorted(jumpResults.keys()):
    jumpResult = jumpResults[resultRef]
    if jumpResult.score > 0.0:
        validJumpStatus = '<hr><h1><span style="color: %s">%s jump - %s - score = %.02f km/h</span></h1>' % (jumpResult.color, resultRef, jumpResult.result, jumpResult.score)        
    else:
        validJumpStatus = '<hr><h1><span style="color: %s">%s jump - %s</span></h1>' % (jumpResult.color, resultRef, jumpResult.result)
    display(widgets.HTML(validJumpStatus))

    maxSpeed = jumpResult.maxSpeed
    window = jumpResult.window
    mixColor = (mixColor+1)%len(SPEED_COLORS)
    if jumpResult.score > 0.0:
        display(widgets.HTML('<h3>Max speed = {0:,.0f}; '.format(maxSpeed)+('exit at %d m (%d ft), end scoring window at %d m (%d ft)</h3>'%(window.start, 3.2808*window.start, window.end, 3.2808*window.end))))
        # TODO: Move this to an in-notebook function or module.
        displayJumpDataIn(jumpResult.table)
        individualPlot = initializePlot(resultRef)
        individualPlot = initializeExtraYRanges(individualPlot, startY = min(jumpResult.data.altitudeASLFt)-500.0, endY = max(jumpResult.data.altitudeASLFt)+500.0)
        graphAltitude(individualPlot, jumpResult)
        graphAngle(individualPlot, jumpResult)
        hoverValue = bm.HoverTool(tooltips = [ ('Y-val', '@y{0.00}'), ])
        individualPlot.add_tools(hoverValue)
        graphJumpResult(individualPlot,
                        jumpResult,
                        lineColor = SPEED_COLORS[0])
        graphJumpResult(allJumpsPlot,
                        jumpResult,
                        lineColor = SPEED_COLORS[mixColor],
                        legend = '%s - %.2f' % (resultRef, jumpResult.score),
                        showIt = False)

In [ ]:
jumpResult.data.columns

---
## Competition summary

In [ ]:
display(roundedResults)
display(sumResults)
bp.show(allJumpsPlot)